In [1]:
import models
import text
import utils
import data_utils
import json
import commons
import torch
import numpy as np
from scipy.io.wavfile import write

In [2]:
with open(r'pretrained/config.json', 'r') as f:
    config = json.load(f)

In [3]:
device = 'cpu'

In [4]:
from text.symbols import symbols
net_g = models.SynthesizerTrn(
    len(symbols),
    80,
    config['train']['segment_size'] // config['data']['hop_length'],
    n_speakers=config['data']['n_speakers'],
    mas_noise_scale_initial=0.01,
    noise_scale_delta=2e-6,
    **config['model']).cpu()

256 2
Multi-band iSTFT VITS2


/home/michael/Documents/ITMO/EDLM/vosk-ft-test/.venv2/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [5]:
from train_finetune_QAT_everything_2 import prepare_model_for_qat
net_g = prepare_model_for_qat(net_g)

/home/michael/Documents/ITMO/EDLM/vosk-ft-test/.venv2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[QAT] Starting QAT fine-tuning of DECODER only (backend=fbgemm)
[QAT] Removing weight norm from FULL net_g...


/home/michael/Documents/ITMO/EDLM/vosk-ft-test/.venv2/lib/python3.12/site-packages/torch/ao/quantization/observer.py:246: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


[QAT] QAT is ENABLED for net_g.dec (convs/convT/linear via QuantWrapper).


In [6]:
from torch.ao.quantization import convert

net_g = convert(net_g.eval(), inplace=False)

/tmp/ipykernel_88413/4058327776.py:3: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  net_g = convert(net_g.eval(), inplace=False)
/home/michael/Documents/ITMO/EDLM/vosk-ft-test/.venv2/lib/python3.12/site-packages/torch/ao/quantization/utils.py:409: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


In [7]:
ckpt = torch.load("db-finetune/out/G_QAT_260.pth", map_location=device)
missing, unexpected = net_g.load_state_dict(ckpt["model"], strict=False)
print("Missing keys:", missing)
print("Unexpected keys:", unexpected)

net_g.eval()

/home/michael/Documents/ITMO/EDLM/vosk-ft-test/.venv2/lib/python3.12/site-packages/torch/_utils.py:444: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


Missing keys: []
Unexpected keys: []


SynthesizerTrn(
  (enc_p): TextEncoder(
    (emb): Embedding(62, 192)
    (encoder): Encoder(
      (drop): Dropout(p=0.1, inplace=False)
      (attn_layers): ModuleList(
        (0): MultiHeadAttention(
          (conv_q): QuantWrapper(
            (quant): Quantize(scale=tensor([0.1033]), zero_point=tensor([70]), dtype=torch.quint8)
            (dequant): DeQuantize()
            (module): QuantizedConv1d(192, 192, kernel_size=(1,), stride=(1,), scale=0.1287403106689453, zero_point=47)
          )
          (conv_k): QuantWrapper(
            (quant): Quantize(scale=tensor([0.1033]), zero_point=tensor([70]), dtype=torch.quint8)
            (dequant): DeQuantize()
            (module): QuantizedConv1d(192, 192, kernel_size=(1,), stride=(1,), scale=0.04974838346242905, zero_point=64)
          )
          (conv_v): QuantWrapper(
            (quant): Quantize(scale=tensor([0.1033]), zero_point=tensor([70]), dtype=torch.quint8)
            (dequant): DeQuantize()
            (module): Qu

In [8]:
txt = 'Я модель, загруженная из чекп+оинта после квантизации. Если меня слышно, то тот, кто меня квантизов+ал -- достоин уважения.'
out = 'congrats_qat_quanted'

In [9]:
def get_text(txt, config):
    text_norm = text.text_to_sequence_g2p(txt)
    if config['data']['add_blank']:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    print(text_norm)
    return text_norm

In [10]:
def vcss(out, inputstr, i):  # single
    device = torch.device("cpu")  # ВАЖНО: quantized модель = CPU
    net_g.to(device)
    net_g.eval()

    stn_tst = get_text(inputstr, config)

    speed = 1.0
    output_dir = r'outputs'
    sid = torch.LongTensor([i]).to(device)

    with torch.no_grad():
        x_tst = stn_tst.to(device).unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)

        o, o_mb, *_ = net_g.infer(
            x_tst,
            x_tst_lengths,
            sid=sid,
            noise_scale=.667,
            noise_scale_w=0.8,
            length_scale=1 / speed,
        )

        audio = o[0, 0].cpu().numpy() * 32768.0  # vol scale

    write(rf'{output_dir}/{out}.wav', config['data']['sampling_rate'], audio.astype(np.int16))
    print(rf'{out}.wav Generated!')


In [11]:
from torch.nn.utils.weight_norm import WeightNorm, remove_weight_norm


def strip_weight_norm_hooks(model):
    for m in model.modules():
        # Try to remove weight_norm parametrization if present
        try:
            remove_weight_norm(m)
        except (ValueError, AttributeError):
            pass
        # Drop any leftover WeightNorm forward pre-hooks (quantized convs have no weight_g)
        if hasattr(m, "_forward_pre_hooks"):
            for k, hook in list(m._forward_pre_hooks.items()):
                if isinstance(hook, WeightNorm):
                    del m._forward_pre_hooks[k]


# strip_weight_norm_hooks(net_g)

In [12]:
vcss(out, txt, 1)

tensor([ 1,  0, 32,  0, 14,  0,  3,  0, 37,  0, 41,  0, 21,  0, 23,  0, 36,  0,
         8,  0,  3,  0, 59,  0, 14,  0, 26,  0, 45,  0, 54,  0, 60,  0, 22,  0,
        39,  0, 39,  0, 14,  0, 32,  0, 14,  0,  3,  0, 31,  0, 59,  0,  3,  0,
        19,  0, 22,  0, 33,  0, 43,  0, 42,  0, 30,  0, 39,  0, 51,  0, 14,  0,
         3,  0, 43,  0, 41,  0, 47,  0, 36,  0, 23,  0,  3,  0, 33,  0, 55,  0,
        14,  0, 39,  0, 52,  0, 30,  0, 59,  0, 15,  0, 18,  0, 30,  0, 30,  0,
        10,  0,  3,  0, 32,  0, 23,  0, 47,  0, 36,  0, 30,  0,  3,  0, 38,  0,
        22,  0, 40,  0, 15,  0,  3,  0, 47,  0, 35,  0, 58,  0, 49,  0, 39,  0,
        41,  0,  8,  0,  3,  0, 51,  0, 41,  0,  3,  0, 51,  0, 42,  0, 51,  0,
         8,  0,  3,  0, 33,  0, 51,  0, 42,  0,  3,  0, 38,  0, 22,  0, 40,  0,
        15,  0,  3,  0, 33,  0, 55,  0, 14,  0, 39,  0, 52,  0, 30,  0, 59,  0,
        41,  0, 55,  0, 15,  0, 35,  0,  3,  0,  3,  0, 20,  0, 41,  0, 47,  0,
        51,  0, 42,  0, 30,  0, 39,  0, 

Проверяем, что модули стали квантизованными

In [13]:
import torch.nn.quantized as nnq

def print_quantized_modules(model):
    for name, m in model.named_modules():
        if isinstance(m, (nnq.Conv1d, nnq.Conv2d, nnq.Conv3d,
                          nnq.ConvTranspose1d, nnq.ConvTranspose2d,
                          nnq.Linear)):
            print("Q MODULE:", name, "->", type(m))

print_quantized_modules(net_g)


Q MODULE: enc_p.encoder.attn_layers.0.conv_q.module -> <class 'torch.ao.nn.quantized.modules.conv.Conv1d'>
Q MODULE: enc_p.encoder.attn_layers.0.conv_k.module -> <class 'torch.ao.nn.quantized.modules.conv.Conv1d'>
Q MODULE: enc_p.encoder.attn_layers.0.conv_v.module -> <class 'torch.ao.nn.quantized.modules.conv.Conv1d'>
Q MODULE: enc_p.encoder.attn_layers.0.conv_o.module -> <class 'torch.ao.nn.quantized.modules.conv.Conv1d'>
Q MODULE: enc_p.encoder.attn_layers.1.conv_q.module -> <class 'torch.ao.nn.quantized.modules.conv.Conv1d'>
Q MODULE: enc_p.encoder.attn_layers.1.conv_k.module -> <class 'torch.ao.nn.quantized.modules.conv.Conv1d'>
Q MODULE: enc_p.encoder.attn_layers.1.conv_v.module -> <class 'torch.ao.nn.quantized.modules.conv.Conv1d'>
Q MODULE: enc_p.encoder.attn_layers.1.conv_o.module -> <class 'torch.ao.nn.quantized.modules.conv.Conv1d'>
Q MODULE: enc_p.encoder.attn_layers.2.conv_q.module -> <class 'torch.ao.nn.quantized.modules.conv.Conv1d'>
Q MODULE: enc_p.encoder.attn_layers.2